In [5]:
import torch

from flair.data import Sentence
from flair.embeddings import DocumentPoolEmbeddings, FlairEmbeddings, BertEmbeddings

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [8]:
gold_filename = "goldstandard_eu.tsv"

def get_gold_sentences(filename):
    gold_sentences = {}
    with open(filename, 'rt') as f_p:
        for line in f_p:
            if line.startswith('"origin"'): # header
                continue
            
            if not line:
                continue
            
            line = line.rstrip()
            line = line.replace('"', '')
            splitted = line.split('\t')
            gold = splitted[0]
            sim_sentences = splitted[1:11]
            
            if gold:
                gold_sentences[gold] = sim_sentences
            
    return gold_sentences

In [9]:
gold_sentences = get_gold_sentences(gold_filename)

In [10]:
def calculate_similarities(gold, sim_sentences, embeddings):
    
    similarities = []
    query = gold

    q = Sentence(query)
    embeddings.embed(q)
    
    for sentence in sim_sentences:
        
        s = Sentence(sentence)
        embeddings.embed(s)

        assert q.embedding.shape == s.embedding.shape
        
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        prox = cos(q.embedding, s.embedding)
    
        similarities.append(round(prox.item(), 4))
        
    return similarities

def print_similarities(gold, sim_sentences, similarities):
        
    print(f"Example: {gold}")
    for sentence, similarity in zip(sim_sentences, similarities) :
        print(f"{sentence} - {similarity}")
    
    print("")

In [11]:
# load flair embeddings
flair_embeddings = DocumentPoolEmbeddings([FlairEmbeddings('eu-forward'), 
                                         FlairEmbeddings('eu-backward')
                                        ])

In [12]:
# load BERT embeddings
# See BERT paper, section 5.3 and table 7
bert_layers = '-1,-2,-3,-4'

bert_cased_embeddings = DocumentPoolEmbeddings([BertEmbeddings('bert-base-multilingual-cased',
                                                               layers=bert_layers)])

bert_uncased_embeddings = DocumentPoolEmbeddings([BertEmbeddings('bert-base-multilingual-uncased',
                                                                 layers=bert_layers)])


2019-06-27 19:15:38,021 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [13]:
# load ELMo and ELMo multilingual embeddings

In [14]:
def calculate_and_print(embeddings, similarities_all):
    i = 0
    for gold, sim_sentences in gold_sentences.items():
        similarities = calculate_similarities(gold, sim_sentences, embeddings)
        print_similarities(gold, sim_sentences, similarities)
        similarities_all[i].append(similarities)
        i += 1
    return similarities_all

In [15]:
similarities_all = []
for i in range(10):
    similarities_all.append([])

In [16]:
# calculate similarities based on flair embeddings
similarities_all = calculate_and_print(flair_embeddings, similarities_all)

Example: Jokinek Amaia bazkari batera gonbidatu zuen
Jokinek gonbidapen bat eskeini zion Amaiari bazkari baterako - 0.7616
Jokinek Amaiari galdetu zion, ea berarekin bazkaldu nahi zuen - 0.708
Jokinek Amaiari galdetu zion: Nahiko zenuke nirekin bazkaltzera joan? - 0.5921
Amaiak galdera jaso zuen Jokinengandik, ea bazkaltzea joan nahiko lukeen - 0.5961
Jokinek Amaiari gonbidapen bat eskeini zion otordu baterako - 0.7694
Amaiak gonbidapen bat jaso zuen Jokinengandik - 0.7276
Bazkaltzeko elkartu ziren Jokin eta Amaia - 0.5851
Jokinek eta Amaiak otordu bat gozatu zuten elkarrekin - 0.8053
Amaiak eta Jokinek erabaki zuten, elkarrekin bazkaltzera joatea - 0.6851
Amaiak Jokin gonbidatu zuen bazkari batera - 0.8692

Example: Sagar bat soberan baduzu, asko eskertuko nuke
Emango al zeniezadake soberan duzun sagar bat? - 0.7304
Eskertuko nuke soberan duzun sagar hori emango bazeniezadake - 0.8085
Sagar hori ez baduzu behar, asko eskertuko nuke - 0.8962
Oso pozik negoke sagar bat emango bazeniezad

In [17]:
# calculate similarities based on bert uncased embeddings
similarities_all = calculate_and_print(bert_uncased_embeddings, similarities_all)

Example: Jokinek Amaia bazkari batera gonbidatu zuen
Jokinek gonbidapen bat eskeini zion Amaiari bazkari baterako - 0.9573
Jokinek Amaiari galdetu zion, ea berarekin bazkaldu nahi zuen - 0.9379
Jokinek Amaiari galdetu zion: Nahiko zenuke nirekin bazkaltzera joan? - 0.9066
Amaiak galdera jaso zuen Jokinengandik, ea bazkaltzea joan nahiko lukeen - 0.9212
Jokinek Amaiari gonbidapen bat eskeini zion otordu baterako - 0.948
Amaiak gonbidapen bat jaso zuen Jokinengandik - 0.9407
Bazkaltzeko elkartu ziren Jokin eta Amaia - 0.9377
Jokinek eta Amaiak otordu bat gozatu zuten elkarrekin - 0.9364
Amaiak eta Jokinek erabaki zuten, elkarrekin bazkaltzera joatea - 0.9138
Amaiak Jokin gonbidatu zuen bazkari batera - 0.9745

Example: Sagar bat soberan baduzu, asko eskertuko nuke
Emango al zeniezadake soberan duzun sagar bat? - 0.9373
Eskertuko nuke soberan duzun sagar hori emango bazeniezadake - 0.9371
Sagar hori ez baduzu behar, asko eskertuko nuke - 0.9496
Oso pozik negoke sagar bat emango bazeniezad

In [18]:
# calculate similarities based on bert cased embeddings
similarities_all = calculate_and_print(bert_cased_embeddings, similarities_all)

Example: Jokinek Amaia bazkari batera gonbidatu zuen
Jokinek gonbidapen bat eskeini zion Amaiari bazkari baterako - 0.9656
Jokinek Amaiari galdetu zion, ea berarekin bazkaldu nahi zuen - 0.918
Jokinek Amaiari galdetu zion: Nahiko zenuke nirekin bazkaltzera joan? - 0.8871
Amaiak galdera jaso zuen Jokinengandik, ea bazkaltzea joan nahiko lukeen - 0.9011
Jokinek Amaiari gonbidapen bat eskeini zion otordu baterako - 0.9612
Amaiak gonbidapen bat jaso zuen Jokinengandik - 0.927
Bazkaltzeko elkartu ziren Jokin eta Amaia - 0.9237
Jokinek eta Amaiak otordu bat gozatu zuten elkarrekin - 0.9315
Amaiak eta Jokinek erabaki zuten, elkarrekin bazkaltzera joatea - 0.9056
Amaiak Jokin gonbidatu zuen bazkari batera - 0.9716

Example: Sagar bat soberan baduzu, asko eskertuko nuke
Emango al zeniezadake soberan duzun sagar bat? - 0.9193
Eskertuko nuke soberan duzun sagar hori emango bazeniezadake - 0.9343
Sagar hori ez baduzu behar, asko eskertuko nuke - 0.9557
Oso pozik negoke sagar bat emango bazeniezada

In [19]:
init_notebook_mode(connected=True)

In [26]:
fig = tools.make_subplots(rows=1, cols=10)

for i in range(10):
    print(list(gold_sentences.keys())[i] + '\n')
    print("\n".join(gold_sentences[list(gold_sentences.keys())[i]]))
    trace = go.Heatmap(z=similarities_all[i], y=['Flair', 'BERT uncased', 'BERT cased'])
    data=[trace]
    fig.append_trace(trace, 1, i+1)
    iplot(data, filename='basic-heatmap' + str(i))

This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]     [ (1,4) x4,y4 ]     [ (1,5) x5,y5 ]     [ (1,6) x6,y6 ]     [ (1,7) x7,y7 ]     [ (1,8) x8,y8 ]     [ (1,9) x9,y9 ]     [ (1,10) x10,y10 ]

Jokinek Amaia bazkari batera gonbidatu zuen

Jokinek gonbidapen bat eskeini zion Amaiari bazkari baterako
Jokinek Amaiari galdetu zion, ea berarekin bazkaldu nahi zuen
Jokinek Amaiari galdetu zion: Nahiko zenuke nirekin bazkaltzera joan?
Amaiak galdera jaso zuen Jokinengandik, ea bazkaltzea joan nahiko lukeen
Jokinek Amaiari gonbidapen bat eskeini zion otordu baterako
Amaiak gonbidapen bat jaso zuen Jokinengandik
Bazkaltzeko elkartu ziren Jokin eta Amaia
Jokinek eta Amaiak otordu bat gozatu zuten elkarrekin
Amaiak eta Jokinek erabaki zuten, elkarrekin bazkaltzera joatea
Amaiak Jokin gonbidatu zuen bazkari batera


Sagar bat soberan baduzu, asko eskertuko nuke

Emango al zeniezadake soberan duzun sagar bat?
Eskertuko nuke soberan duzun sagar hori emango bazeniezadake
Sagar hori ez baduzu behar, asko eskertuko nuke
Oso pozik negoke sagar bat emango bazeniezadake
Emango didazu sagar bat?
Sagar hori nahi nuke
Sagar bat eskatu dizut
Behar baino sagar bat gehiago duzu
Sagar bat eskeini didazu
Sagar bat oparitu didazu


Anek Amaia zoriondu du

Anek Amaiari zorionak esan dio
Anek Amaia goraipatu du
Zoriondua izan da Amaia Anerengandik
Zoriona opa diola esan dio Anek Amaiari
Anek Amaiari urte askotarako esan dio
Amaiaren urtebetetzea dela eta, zorion asko jaso ditu
Opari bat eman dio Anek Amaiari
Amaia eta Ane elkar zoriondu dira
Zorionduak izan dira Ane eta Amaia
Amaiak Aneri zorionak esan dio


Mikel mendira joan da

Mendira joan da Mikel
Mendian gora abiatu da Mikel
Mendian egon da Mikel
Mikelek erabaki du mendira joatea
Mendira joateko plana egin du Mikelek
Mikelek esan du, mendira doala
Mikelek aipamena egin du, agian mendira joango zela
Mikelek mendia gustoko du
Mikelek esan du, ez doala mendira
Mikel ez da mendira joan


Atzo gure etxera etorri zinen afaltzera

Afaltzera etorri zinen gure etxera atzo
Atzo gurekin afaldu zenuen gure etxean
Gure etxean izan zinen atzo afaltzen
Atzo gure etxean afaltzeko gonbidapena onartu zenuen
Atzo elkarrekin afaldu genuen denok
Elkarrekin bazkaldu genezakeen atzo
Atzo etorri zen afaltzera
Bihar etorriko da bazkaltzera
Zure etxera joan nitzen afaltzera
Atzo zuen etxera etorriko dira afaltzera


Zoriontasuna ez da norberak gustoko duena egitea, egiten duzuna gustokoa izatea baizik

Egiten duzuna gustokoa izateak egiten zaitu zoriontsu, ez zuk nahi duzuna egiteak
Izan ditzazu gustoko egindako gauzak eta izango zara zoriontsu
Zoriontasunaren iturria da egiten duzuna maitatzea
Zoriontasuna da, egindakoa gustoko izatea
Gustuz egiten dena da zoriontasuna
Gustoko duzuna egiteak ez zaitu zoriontsu egingo
Egiten duzuna, ez gustoko duzuna, da zoriontasuna
Egin nahi duzuna zoriontsu izateko
Egin ditzazu gustoko dituzun gauzak eta izango zara zoriontsu
Beti ezin da norberak gustoko duena egin


Kolorea eman mundu grisari

Eman kolorea mundu grisari
Mundu grisari emaiozu kolorea
Mundua grisa dela eta eman diezaiogun kolorea
Kolorea eman mundu ilunari
Planetaren gristasuna ezabatzeko kolorea behar du
Mundua grisa denez, kolorea emango diogu
Eman kolorea munduari
Grisa eman mundu koloretsuari
Eman grisa munduari
Mundua eman kolore grisari


Maite dudalako sortzen dut

Sortzen dut maite dudalako
Maitasunaren ondorioz sortzen dut
Maite dut: ondorioz, sortzen dut
Sortzearen arrazoia maitasuna da
Nire sorkuntzaren iturria maitasuna da
Maite nuelako sortu nuen
Sortu ondoren maitatu nuen
Maite duzulako sortzen duzu
Ez dut maite, beraz ez dut sortzen
Sortzen dudalako maite dut


Jakingo bazenu zein polita den zu euskaraz entzutea

Oso polita da zu euskaraz entzutea
Zu euskaraz entzuteak asko pozten nau
Ez dakizu zein polita den zure euskara entzutea
Ezin azalgarria da nolako zoragarria den zuk euskaraz hitz egiten duzunean
Ezin duzu jakin nolako polita den zu entzutea
Zu euskaraz hitz egitea zein polita den jakitea ezinezkoa da
Zu entzutea zoragarria da
Zu euskaraz entzutea zein polita den jakitea nahiko nuke
Jakingo bazenu zein itsusia den zu euskaraz entzutea
Jakingo balute zein polita den nik euskeraz hitz egitea


IndexError: list index out of range